# GENERAL INFORMATION ABOUT THE DATA

There are 939 unique stops
There are 996250 records in the provided dataset(they in fact represent connections)

Each node in the graph will represent station

# NOTES
Notes: 
1.Include waiting time in the algorithm. E.g if at 8:40 shortly I will have train at 8:42, which takes 10 minutes and bus at 8:45 which takes 6 minutes : waiting time from 8:40 should be included
    
2.What about stations which have the same name, but are in fact two separate stations( Dworzec Głowny
Dworcowa for example). There are two of them but there are different connections from each of them

Since there can be multiple stations with the same name I can use latitude and longitude for it and then even though stations will have the same name it will be different stations (but then the question is if the user wants to select from each station he wants to go, how to distinguish them?) 



In [1]:
# imports

import pandas as pd
import numpy as np
import networkx as nx
from heapq import heappop, heappush
from itertools import count

from datetime import datetime, timedelta


In [2]:
file_path = "./resources/connection_graph.csv"
# Specify data types using dtype parameter
column_data_types = {'line': str}  
df = pd.read_csv(file_path,dtype=column_data_types)


In [3]:
df.rename(columns={'Unnamed: 0': 'id'}, inplace=True)

In [4]:
df.head()

,id,company,line,departure_time,arrival_time,start_stop,end_stop,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
0,0,MPK Autobusy,A,20:52:00,20:53:00,Zajezdnia Obornicka,Paprotna,51.148737,17.021069,51.147752,17.020539
1,1,MPK Autobusy,A,20:53:00,20:54:00,Paprotna,Obornicka (Wołowska),51.147752,17.020539,51.144385,17.023735
2,2,MPK Autobusy,A,20:54:00,20:55:00,Obornicka (Wołowska),Bezpieczna,51.144385,17.023735,51.141360,17.026376
3,3,MPK Autobusy,A,20:55:00,20:57:00,Bezpieczna,Bałtycka,51.141360,17.026376,51.136632,17.030617
4,4,MPK Autobusy,A,20:57:00,20:59:00,Bałtycka,Broniewskiego,51.136632,17.030617,51.135851,17.037383


In [5]:
df.columns

Index(['id', 'company', 'line', 'departure_time', 'arrival_time', 'start_stop',
       'end_stop', 'start_stop_lat', 'start_stop_lon', 'end_stop_lat',
       'end_stop_lon'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 996520 entries, 0 to 996519
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              996520 non-null  int64  
 1   company         996520 non-null  object 
 2   line            996520 non-null  object 
 3   departure_time  996520 non-null  object 
 4   arrival_time    996520 non-null  object 
 5   start_stop      996520 non-null  object 
 6   end_stop        996520 non-null  object 
 7   start_stop_lat  996520 non-null  float64
 8   start_stop_lon  996520 non-null  float64
 9   end_stop_lat    996520 non-null  float64
 10  end_stop_lon    996520 non-null  float64
dtypes: float64(4), int64(1), object(6)
memory usage: 83.6+ MB


In [7]:
df.describe()

,id,start_stop_lat,start_stop_lon,end_stop_lat,end_stop_lon
count,996520.000000,996520.000000,996520.000000,996520.000000,996520.000000
mean,498259.500000,51.114712,17.017362,51.114711,17.017354
std,287670.689464,0.027226,0.060351,0.027224,0.060350
min,0.000000,50.980426,16.695853,50.979741,16.695853
25%,249129.750000,51.097100,16.978353,51.097100,16.978353
50%,498259.500000,51.113789,17.024670,51.113789,17.024670
75%,747389.250000,51.135096,17.050671,51.135096,17.050671
max,996519.000000,51.265566,17.285545,51.265566,17.285545


In [8]:
df.shape

(996520, 11)

### Check if the data is mirrored ( if there is the same number of records of shape 27:35:00 as 3:35:00)


In [9]:
# Check the number of records with departure_time equal to '27:35:00'
count_records_273500 = df[df['departure_time'] == '27:35:00'].shape[0]
print(f"Number of records with departure_time '27:35:00': {count_records_273500}")

# Check the number of records with arrival_time equal to '27:35:00'
count_records_arrival_273500 = df[df['arrival_time'] == '27:35:00'].shape[0]
print(f"Number of records with arrival_time '27:35:00': {count_records_arrival_273500}")

# Check the number of records with departure_time equal to '03:35:00'
count_records_033500 = df[df['departure_time'] == '03:35:00'].shape[0]
print(f"Number of records with departure_time '03:35:00': {count_records_033500}")

# Check the number of records with arrival_time equal to '03:35:00'
count_records_arrival_033500 = df[df['arrival_time'] == '03:35:00'].shape[0]
print(f"Number of records with arrival_time '03:35:00': {count_records_arrival_033500}")


Number of records with departure_time '27:35:00': 96
Number of records with arrival_time '27:35:00': 96
Number of records with departure_time '03:35:00': 0
Number of records with arrival_time '03:35:00': 0


In [10]:
all_station_names = pd.unique(df[['start_stop', 'end_stop']].values.ravel('K'))
all_station_names

array(['Zajezdnia Obornicka', 'Paprotna', 'Obornicka (Wołowska)',
       'Bezpieczna', 'Bałtycka', 'Broniewskiego', 'Pola', 'Syrokomli',
       'Kasprowicza', 'pl. Daniłowskiego', 'Przybyszewskiego',
       'Czajkowskiego', 'KOSZAROWA (Uniwersytet)', 'KOSZAROWA (Szpital)',
       'Berenta', 'KROMERA', 'Mosty Warszawskie', 'Wyszyńskiego',
       'Ogród Botaniczny', 'Katedra',
       'Urząd Wojewódzki (Muzeum Narodowe)', 'Poczta Główna',
       'skwer Krasińskiego', 'Wzgórze Partyzantów', 'Renoma',
       'Arkady (Capitol)', 'Pl. Hirszfelda', 'Krucza',
       'Krucza (Mielecka)', 'Inżynierska', 'Aleja Pracy', 'FAT',
       'GRABISZYŃSKA (Cmentarz)', 'Solskiego', 'Wiejska', 'Kadłubka',
       'Stanki', 'Bukowskiego', 'RACŁAWICKA', 'Rymarska', 'Wawrzyniaka',
       'Chłodna', 'Sowia', 'KRZYKI', 'Dworzec Główny (Dworcowa)',
       'Krasińskiego', 'Damrota', 'Koszarowa', 'Sołtysowicka',
       'Poprzeczna', 'Redycka', 'Bagatela', 'SOŁTYSOWICE', 'POŚWIĘTNE',
       'Wołowska', 'Kępińska', 'Ka

In [11]:
len(all_station_names)

939

In [12]:
unique_start_stops = df['start_stop'].nunique()
unique_end_stops = df['end_stop'].nunique()


print(f"Number of unique names in 'start_stop': {unique_start_stops}")
print(f"Number of unique names in 'end_stop': {unique_end_stops}")

Number of unique names in 'start_stop': 938
Number of unique names in 'end_stop': 937


In [13]:
start_stop_unique = set(df['start_stop'].unique())
end_stop_unique = set(df['end_stop'].unique())

difference_elements = start_stop_unique - end_stop_unique

print("Elements in start_stop but not in end_stop:", difference_elements)


Elements in start_stop but not in end_stop: {'Sucha', 'Żórawina - kościół Św. Trójcy'}


In [14]:
# verify
sucha_as_end_stop_rows = df[df['end_stop'] == 'Sucha']

## FIND UNIQUE STOPS BASED ON LOCATION


So I was thinking to represent two stations(in different directions) as separate nodes, but turns that there can be more than 2(as on Dworcowa that there can be some substation where only specific buses stop). So I will just treat all these different sub stations as single one. So I will have single which will have all these connections( all connections from 4 of them)



In [15]:
start_stops = df[['start_stop', 'start_stop_lat', 'start_stop_lon']].copy()
start_stops = start_stops.rename(columns={'start_stop': 'station_name', 'start_stop_lat': 'station_lat', 'start_stop_lon': 'station_lon'})

end_stops = df[['end_stop', 'end_stop_lat', 'end_stop_lon']].copy()
end_stops = end_stops.rename(columns={'end_stop': 'station_name', 'end_stop_lat': 'station_lat', 'end_stop_lon': 'station_lon'})

all_stops = pd.concat([start_stops, end_stops], ignore_index=True)
unique_stops = all_stops.drop_duplicates(subset=['station_lat', 'station_lon'])


In [16]:
dworcowa_stops = unique_stops[unique_stops['station_name'].str.contains('Dworcowa')]
dworcowa_stops

,station_name,station_lat,station_lon
63,Dworzec Główny (Dworcowa),51.101149,17.040017
455681,Dworzec Główny (Dworcowa),51.100278,17.039201
533787,Dworzec Główny (Dworcowa),51.101064,17.040380
856943,Dworzec Główny (Dworcowa),51.100601,17.039271
951999,Smolec - Główna/Dworcowa,51.072252,16.880387


In [17]:
most_grunwaldzki_stops = unique_stops[unique_stops['station_name'].str.contains('most Grunwaldzki')]
most_grunwaldzki_stops

,station_name,station_lat,station_lon
14028,most Grunwaldzki,51.110136,17.055093
14160,most Grunwaldzki,51.110133,17.055573
97771,most Grunwaldzki,51.110212,17.055385
97811,most Grunwaldzki,51.110138,17.055493


# THIS IS THE DATA WHICH WILL BE USED TO CREATE NODES IN MY DIRECTED GRAPH(THEY REPRESENT STATIONS)

### Since there are multiple with the same name ( and in fact they are in different locations, but still close) to represent it i will take average lon and lat for stations with the same name

In [18]:
# Group by 'station_name' and calculate the average 'stop_lat' and 'stop_lon'
avg_stops = unique_stops.groupby('station_name').agg({'station_lat': 'mean', 'station_lon': 'mean'}).reset_index()

# Display the final DataFrame with average values
print(avg_stops)

                         station_name  station_lat  station_lon
0                              8 Maja    51.113536    17.091181
1                              AUCHAN    51.052065    16.974054
2                       Adamczewskich    51.120899    16.869548
3                        Adamieckiego    51.073771    16.963324
4                   Aleja Architektów    51.138955    16.928764
..                                ...          ...          ...
934             Żórawina - Wrocławska    50.986326    17.038462
935     Żórawina - kościół Św. Trójcy    50.980426    17.039577
936                Żórawina - osiedle    50.987617    17.012400
937                 Żórawina - skrzy.    50.981035    17.035511
938  Żórawina - skrzy. Niepodległości    50.981291    17.035642

[939 rows x 3 columns]


## HERE THE PROBLEM IS THAT DATA IS LIKE 24:01:00 AND TIME GOES BEYOND 24 HOURS. SO HERE I AM CHANGING THE FORMAT OF DATA TO 24 HOUR TIME

### PREPARE THE DATA FOR THE EDGES(CONNECTIONS) BETWEEN STATIONS

In [19]:
columns_to_extract = ['line','start_stop', 'end_stop', 'departure_time', 'arrival_time']
connections = df[columns_to_extract].copy()

# Convert 'departure_time' and 'arrival_time' to timedelta objects
connections['departure_seconds'] = pd.to_timedelta(connections['departure_time']).dt.total_seconds()
connections['arrival_seconds'] = pd.to_timedelta(connections['arrival_time']).dt.total_seconds()

# Handle values exceeding 24 hours using modulo operator
connections['departure_seconds'] = connections['departure_seconds'] % (24 * 3600)
connections['arrival_seconds'] = connections['arrival_seconds'] % (24 * 3600)

# Convert back to time objects
connections['departure_time'] = pd.to_datetime(connections['departure_seconds'], unit='s').dt.time
connections['arrival_time'] = pd.to_datetime(connections['arrival_seconds'], unit='s').dt.time

# Drop intermediate columns
connections.drop(['departure_seconds', 'arrival_seconds'], axis=1, inplace=True)






### NOW I WILL CALCULATE TIME DIFFERENCE BETWEEN CONNECTIONS IN TIME. SINCE THERE CAN BE CASE WHEN departure_time is 23:59:00 and arrival_time is 00:01:00 if i subtract arrival_time from departure_time I will get negative value, so I need to handle such cases, because I want to get time difference as 2. 

!!! SINCE THE WAITING SHOULD BE INCLUDED INTO CALCULATIONS THERE IS NO POINT TO CALCULATE TIME DIFFERENCE COLUMN UPFRONT



In [20]:
connections.head()

,line,start_stop,end_stop,departure_time,arrival_time
0,A,Zajezdnia Obornicka,Paprotna,20:52:00,20:53:00
1,A,Paprotna,Obornicka (Wołowska),20:53:00,20:54:00
2,A,Obornicka (Wołowska),Bezpieczna,20:54:00,20:55:00
3,A,Bezpieczna,Bałtycka,20:55:00,20:57:00
4,A,Bałtycka,Broniewskiego,20:57:00,20:59:00


### Verify whether conversion to datetime.time object was successful

In [21]:
import datetime

desired_time = datetime.time(23, 59, 0)

desired_records = connections[connections['departure_time'] == desired_time]

desired_records

,line,start_stop,end_stop,departure_time,arrival_time
30968,K,Kamieńskiego,Broniewskiego,23:59:00,00:01:00
32994,K,Kamieńskiego,Broniewskiego,23:59:00,00:01:00
35129,K,Kamieńskiego,Broniewskiego,23:59:00,00:01:00
38108,K,Kamieńskiego,Broniewskiego,23:59:00,00:01:00
41103,N,Kromera (Czajkowskiego),KROMERA,23:59:00,00:00:00
...,...,...,...,...,...
950293,904,Zielna,C.H. Korona,23:59:00,00:01:00
951455,904,Psie Pole,Zielna,23:59:00,23:59:00
951456,904,Zielna,C.H. Korona,23:59:00,00:01:00
955346,908,Żmigrodzka (Obwodnica),Lekarska,23:59:00,00:00:00


In [22]:
# so there are 24 hours right now, which is correct
connections['departure_time'].apply(lambda x: x.hour).nunique()


24

In [23]:
len(connections)

996520

# SO NOW DATA IS READY

## PREPARE HELPER FUNCTIONS WHICH WILL BE USED FOR CALCULATIONS

In [24]:
import datetime

def calculate_time_difference(departure_time, arrival_time):
    # Convert departure_time and arrival_time to seconds
    departure_seconds = departure_time.hour * 3600 + departure_time.minute * 60 + departure_time.second
    arrival_seconds = arrival_time.hour * 3600 + arrival_time.minute * 60 + arrival_time.second
    
    # Calculate the time difference in seconds
    time_difference_seconds = arrival_seconds - departure_seconds
    
    # Handle cases where the difference is negative
    if time_difference_seconds < 0:
        time_difference_seconds += 24 * 60 * 60  # 24 hours in seconds
    
    return time_difference_seconds / 60  # Convert to minutes


In [25]:
# Example test calls
departure_time1 = datetime.time(8, 30, 0)
arrival_time1 = datetime.time(10, 15, 0)
print("Time difference:", calculate_time_difference(departure_time1, arrival_time1), "minutes")

departure_time2 = datetime.time(23, 45, 0)
arrival_time2 = datetime.time(0, 15, 0)  # next day
print("Time difference:", calculate_time_difference(departure_time2, arrival_time2), "minutes")

departure_time3 = datetime.time(18, 0, 0)
arrival_time3 = datetime.time(18, 30, 0)
print("Time difference:", calculate_time_difference(departure_time3, arrival_time3), "minutes")

departure_time3 = datetime.time(18, 0, 0)
arrival_time3 = datetime.time(17, 59, 0)
print("Time difference:", calculate_time_difference(departure_time3, arrival_time3), "minutes")

departure_time3 = datetime.time(8, 20, 0)
arrival_time3 = datetime.time(8, 15, 0)
print("Time difference:", calculate_time_difference(departure_time3, arrival_time3), "minutes")

Time difference: 105.0 minutes
Time difference: 30.0 minutes
Time difference: 30.0 minutes
Time difference: 1439.0 minutes
Time difference: 1435.0 minutes


### PREPARE HEURITICS FUNCTIONS

In [26]:
import math
def euclidean_distance(node1, node2):
    lat1, lon1 = node1['lat'], node1['lon']
    lat2, lon2 = node2['lat'], node2['lon']
    
    # Calculate the Euclidean distance
    distance = math.sqrt((lat2 - lat1)**2 + (lon2 - lon1)**2)
    return distance

def manhattan_distance(node1, node2):
    lat1, lon1 = node1['lat'], node1['lon']
    lat2, lon2 = node2['lat'], node2['lon']
    
    # Calculate the Manhattan distance
    distance = abs(lat2 - lat1) + abs(lon2 - lon1)
    return distance

In [27]:
from datetime import datetime, time

def str_to_time(time_str):
    # Parse the time string and extract the hour, minute, and second components
    hour, minute, second = map(int, time_str.split(':'))
    
    # Return a datetime.time object
    return time(hour, minute, second)

# CREATE GRAPH

In [28]:
import pickle

def save_graph(graph, filename):
    with open(filename, 'wb') as f:
        pickle.dump(graph, f)

def load_graph(filename):
    with open(filename, 'rb') as f:
        graph = pickle.load(f)
    return graph

In [29]:
def add_nodes_from_dataframe(graph, dataframe):
    for index, row in dataframe.iterrows():
        station_name = row['station_name']
        station_lat = row['station_lat']
        station_lon = row['station_lon']
        graph.add_node(station_name, lat=station_lat, lon=station_lon)

In [30]:
def update_transportation_system_edges(graph, connections):
    # Remove existing edges from the graph
    graph.remove_edges_from(graph.edges())
    
    # Add new edges from the connections DataFrame
    for index, row in connections.iterrows():
        line = row['line']
        start_stop = row['start_stop']
        end_stop = row['end_stop']
        
        # Generate a unique identifier for the edge if needed
        edge_id = index  # Or any other method to generate a unique identifier
        
        # Add an edge to the graph connecting start_stop to end_stop with the specified attributes and weight
        graph.add_edge(start_stop, end_stop, departure_time=row['departure_time'], 
                       arrival_time=row['arrival_time'], line=line, edge_id=edge_id)


# Load the graph from the file to not recreate graph each time(since it will not change)

In [31]:
import os
filename = "transportation_system.pkl"
if not os.path.isfile(filename):
    transportation_system = nx.MultiDiGraph()
    add_nodes_from_dataframe(transportation_system,avg_stops)
    update_transportation_system_edges(transportation_system, connections)
    save_graph(transportation_system, filename)
else:
    transportation_system = load_graph(filename)


# VERIFY IF THE DATA IS LOADED CORRECTLY

In [32]:
print("Nodes in the transportation system graph:")
print(list(transportation_system.nodes(data=True))[:5])

Nodes in the transportation system graph:
[('8 Maja', {'lat': 51.11353644166667, 'lon': 17.09118084}), ('AUCHAN', {'lat': 51.0520648, 'lon': 16.97405354}), ('Adamczewskich', {'lat': 51.120898675, 'lon': 16.869548215000002}), ('Adamieckiego', {'lat': 51.07377125, 'lon': 16.963323875}), ('Aleja Architektów', {'lat': 51.138954562500004, 'lon': 16.9287636275})]


In [33]:
num_nodes = transportation_system.number_of_nodes()
print(f"The number of nodes in the graph is: {num_nodes}")


The number of nodes in the graph is: 939


In [34]:
num_edges = transportation_system.number_of_edges()
print(f"The number of edges in the graph is: {num_edges}")


The number of edges in the graph is: 996520


In [35]:
import itertools

print("Subset of edges in the transportation system graph:")
edges_to_display = itertools.islice(transportation_system.edges(data=True), 5)  # Selecting the first five edges

for edge in edges_to_display:
    print(edge)

Subset of edges in the transportation system graph:
('8 Maja', 'Stadion Olimpijski', {'departure_time': datetime.time(10, 20), 'arrival_time': datetime.time(10, 21), 'line': '9', 'edge_id': 167372})
('8 Maja', 'Stadion Olimpijski', {'departure_time': datetime.time(14, 20), 'arrival_time': datetime.time(14, 21), 'line': '9', 'edge_id': 167421})
('8 Maja', 'Stadion Olimpijski', {'departure_time': datetime.time(18, 20), 'arrival_time': datetime.time(18, 21), 'line': '9', 'edge_id': 167470})
('8 Maja', 'Stadion Olimpijski', {'departure_time': datetime.time(10, 40), 'arrival_time': datetime.time(10, 41), 'line': '9', 'edge_id': 167519})
('8 Maja', 'Stadion Olimpijski', {'departure_time': datetime.time(14, 40), 'arrival_time': datetime.time(14, 41), 'line': '9', 'edge_id': 167568})


## CREATE HASH MAP SO IN THE END I CAN RETRIEVE CONNECTIONS BASED ON EDGE_ID (using hash map, so I have got O(1) access time)

In [36]:
edge_hashmap = {}

# Iterate over the edges and populate the hashmap
for edge in transportation_system.edges(data=True):
    u, v, data = edge
    edge_id = data.get('edge_id')
    if edge_id:
        edge_hashmap[edge_id] = edge

In [37]:
#TEST ACCESS
edge_id_to_find = 841052
edge = edge_hashmap.get(edge_id_to_find)
print(edge)

('DWORZEC GŁÓWNY', 'Dworzec Główny (Dworcowa)', {'departure_time': datetime.time(8, 17), 'arrival_time': datetime.time(8, 20), 'line': '145', 'edge_id': 841052})


### THIS IS FUNCTION WHICH IS USED TO DISPLAY FINAL RESULT(MODIFY IF U WANT TO CHANGE HOW THE OUTPOOT LOOKS LIKE)

In [38]:
def display_edges_by_id(edge_list, edge_hashmap, start_time):
    for edge_id_to_find in edge_list:
        edge = edge_hashmap.get(edge_id_to_find)
        if edge:
            print(f"{edge[0]} -> {edge[1]}, {edge[2]['departure_time']} - {edge[2]['arrival_time']}, Line: {edge[2]['line']}")
            end_time = edge[2]['arrival_time']
        else:
            print("Edge with edge_id", edge_id_to_find, "not found.")
     
    if end_time:
        print(f"Time of trip: {start_time} - {end_time}({int(calculate_time_difference(str_to_time(start_time),end_time))} minutes)")
    return end_time


## THIS IS THE ALGORITHM BASED ON TIME


## IMPORTANT FUNCTION WHICH IS USED TO FIND EDGE WITH MINIMAL TIME_DIFFERENCE

In [39]:
def find_minimal_time_difference_edge(w, current_time):
    minimal_difference = float('inf')  # Initialize with infinity
    minimal_difference_key = None
    for key, value in w.items():
        
        arrival_time = value['arrival_time']
        departure_time = value['departure_time']
        waiting_time = calculate_time_difference(current_time, departure_time) # firstly i need to calculate the difference between current_time and departure_time
        travel_time = calculate_time_difference(departure_time,arrival_time)
    
        difference = waiting_time + travel_time    
        # Update minimal_difference if the current difference is smaller
        if difference < minimal_difference:
            minimal_difference = difference
            minimal_difference_key = key
    
    return minimal_difference_key, minimal_difference

# THIS IS ALGORITHM WHICH IS USED TO FIND SHORTEST PATH BASED ON TIME(SHORTEST TIME)

In [40]:
def astar_path(G, source, target,start_time,heuristic=None):
    start_time = str_to_time(start_time)
    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)
    if heuristic is None:
        # The default heuristic is h=0 - same as Dijkstra's algorithm
        def heuristic(u, v):
            return 0
    push = heappush
    pop = heappop

    G_succ = G._adj  # For speed-up (and works for both directed and undirected graphs)

    c = count()
    queue = [(0, next(c), source, 0,None,start_time)] # priority,consecutive_id,current_node,distance,parent,parent_connection,current_time 
    enqueued = {}
    explored = {}
    while queue:
        # Pop the smallest item from queue.
        _, __, curnode, dist,parent_connection, current_time = pop(queue) # current_time specifies at which time we are at particular node

        if curnode == target:
            path = [parent_connection]
            node = edge_hashmap.get(parent_connection)[0]
            while explored[node] is not None:
                path.append(explored[node])
                node = edge_hashmap.get(explored[node])[0]
            path.reverse()
            return path

        if curnode in explored:
            # Do not override the parent of starting node
            if explored[curnode] is None:
                continue
            # Skip bad paths that were enqueued before finding a better one
            qcost, h = enqueued[curnode]
            if qcost < dist:
                continue
        explored[curnode] = parent_connection
        for neighbor, w in G_succ[curnode].items():
        
            minimal_edge,cost = find_minimal_time_difference_edge(w,current_time)
            if minimal_edge is None:
                continue
            ncost = dist + cost
            if neighbor in enqueued:
                qcost, h = enqueued[neighbor]
                # if qcost <= ncost, a less costly path from the
                # neighbor to the source was already determined.
                # Therefore, we won't attempt to push this neighbor
                # to the queue
                if qcost <= ncost:
                    continue
            else:
                h = heuristic(G.nodes[neighbor], G.nodes[target])
            enqueued[neighbor] = ncost, h
            
            push(queue, (ncost + h, next(c), neighbor, ncost,w[minimal_edge]['edge_id'],w[minimal_edge]['arrival_time']))
            
    raise nx.NetworkXNoPath(f"Node {target} not reachable from {source}")

start_time = "08:10:00"
result = astar_path(transportation_system,"Dworzec Główny (Dworcowa)","most Grunwaldzki",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

Dworzec Główny (Dworcowa) -> Kościuszki, 08:11:00 - 08:12:00, Line: 114
Kościuszki -> Komuny Paryskiej, 08:13:00 - 08:14:00, Line: 4
Komuny Paryskiej -> pl. Wróblewskiego, 08:14:00 - 08:16:00, Line: 4
pl. Wróblewskiego -> Urząd Wojewódzki (Impart), 08:16:00 - 08:19:00, Line: 4
Urząd Wojewódzki (Impart) -> most Grunwaldzki, 08:19:00 - 08:20:00, Line: 146
Time of trip: 08:10:00 - 08:20:00(10 minutes)


datetime.time(8, 20)

### TIME SHOULD BE PROVIDED IN SUCH FORMAT:   HOURS:MINUTES:SECONDS

In [41]:
start_time = "08:10:00"
result = astar_path(transportation_system,"Dworzec Główny (Dworcowa)","most Grunwaldzki",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

Dworzec Główny (Dworcowa) -> Kościuszki, 08:11:00 - 08:12:00, Line: 114
Kościuszki -> Komuny Paryskiej, 08:13:00 - 08:14:00, Line: 4
Komuny Paryskiej -> pl. Wróblewskiego, 08:14:00 - 08:16:00, Line: 4
pl. Wróblewskiego -> Urząd Wojewódzki (Impart), 08:16:00 - 08:19:00, Line: 4
Urząd Wojewódzki (Impart) -> most Grunwaldzki, 08:19:00 - 08:20:00, Line: 146
Time of trip: 08:10:00 - 08:20:00(10 minutes)


datetime.time(8, 20)

In [42]:
start_time = "23:45:00"
result = astar_path(transportation_system,"most Grunwaldzki","Tramwajowa",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

most Grunwaldzki -> PL. GRUNWALDZKI, 23:51:00 - 23:53:00, Line: 241
PL. GRUNWALDZKI -> Kliniki - Politechnika Wrocławska, 00:00:00 - 00:02:00, Line: 253
Kliniki - Politechnika Wrocławska -> Hala Stulecia, 00:02:00 - 00:03:00, Line: 253
Hala Stulecia -> Tramwajowa, 00:03:00 - 00:04:00, Line: 253
Time of trip: 23:45:00 - 00:04:00(19 minutes)


datetime.time(0, 4)

In [43]:
start_time = "23:45:00"
result = astar_path(transportation_system,"Dworzec Główny (Dworcowa)","most Grunwaldzki",start_time,euclidean_distance)
display_edges_by_id(result,edge_hashmap,start_time)

Dworzec Główny (Dworcowa) -> Kościuszki, 04:34:00 - 04:35:00, Line: 114
Kościuszki -> Komuny Paryskiej, 04:37:00 - 04:38:00, Line: 4
Komuny Paryskiej -> pl. Wróblewskiego, 04:38:00 - 04:39:00, Line: 4
pl. Wróblewskiego -> Urząd Wojewódzki (Impart), 04:39:00 - 04:41:00, Line: 4
Urząd Wojewódzki (Impart) -> most Grunwaldzki, 04:41:00 - 04:43:00, Line: 4
Time of trip: 23:45:00 - 04:43:00(298 minutes)


datetime.time(4, 43)

In [44]:
start_time = "00:01:00"
result = astar_path(transportation_system,"Tramwajowa","Psie Pole",start_time,euclidean_distance)
display_edges_by_id(result,edge_hashmap,start_time)

Tramwajowa -> ZOO, 00:01:00 - 00:03:00, Line: 10
ZOO -> Hala Stulecia, 00:03:00 - 00:04:00, Line: 10
Hala Stulecia -> Kliniki - Politechnika Wrocławska, 00:04:00 - 00:06:00, Line: 10
Kliniki - Politechnika Wrocławska -> PL. GRUNWALDZKI, 00:06:00 - 00:08:00, Line: 10
PL. GRUNWALDZKI -> Kochanowskiego, 00:12:00 - 00:14:00, Line: 116 
Kochanowskiego -> Śniadeckich, 00:14:00 - 00:15:00, Line: 116 
Śniadeckich -> Zacisze, 00:15:00 - 00:16:00, Line: 116 
Zacisze -> Kwidzyńska, 00:16:00 - 00:18:00, Line: 116 
Kwidzyńska -> KROMERA, 00:18:00 - 00:20:00, Line: 116 
KROMERA -> Kromera (Czajkowskiego), 00:20:00 - 00:21:00, Line: 251
Kromera (Czajkowskiego) -> Grudziądzka, 00:21:00 - 00:22:00, Line: 251
Grudziądzka -> Brücknera, 00:22:00 - 00:23:00, Line: 251
Brücknera -> C.H. Korona, 00:23:00 - 00:24:00, Line: 251
C.H. Korona -> Zielna, 00:24:00 - 00:26:00, Line: 251
Zielna -> Psie Pole, 00:26:00 - 00:27:00, Line: 251
Time of trip: 00:01:00 - 00:27:00(26 minutes)


datetime.time(0, 27)

In [45]:
start_time = "23:50:00"
result = astar_path(transportation_system,"Tramwajowa","Psie Pole",start_time)
display_edges_by_id(result,edge_hashmap,start_time)

Tramwajowa -> ZOO, 23:54:00 - 23:56:00, Line: 4
ZOO -> Hala Stulecia, 23:56:00 - 23:57:00, Line: 4
Hala Stulecia -> Kliniki - Politechnika Wrocławska, 23:57:00 - 23:59:00, Line: 4
Kliniki - Politechnika Wrocławska -> PL. GRUNWALDZKI, 23:59:00 - 00:01:00, Line: 4
PL. GRUNWALDZKI -> Piastowska, 00:01:00 - 00:03:00, Line: 4
Piastowska -> Prusa, 00:03:00 - 00:05:00, Line: 4
Prusa -> Wyszyńskiego, 00:05:00 - 00:06:00, Line: 4
Wyszyńskiego -> Damrota, 00:07:00 - 00:08:00, Line: 115
Damrota -> KROMERA, 00:08:00 - 00:10:00, Line: 115
KROMERA -> Kromera (Czajkowskiego), 00:20:00 - 00:21:00, Line: 251
Kromera (Czajkowskiego) -> Grudziądzka, 00:21:00 - 00:22:00, Line: 251
Grudziądzka -> Brücknera, 00:22:00 - 00:23:00, Line: 251
Brücknera -> C.H. Korona, 00:23:00 - 00:24:00, Line: 251
C.H. Korona -> Zielna, 00:24:00 - 00:26:00, Line: 251
Zielna -> Psie Pole, 00:26:00 - 00:27:00, Line: 251
Time of trip: 23:50:00 - 00:27:00(37 minutes)


datetime.time(0, 27)

In [46]:
start_time = "13:01:00"
result = astar_path(transportation_system,"Tramwajowa","Psie Pole",start_time)
display_edges_by_id(result,edge_hashmap,start_time)

Tramwajowa -> Hala Stulecia, 13:01:00 - 13:03:00, Line: 146
Hala Stulecia -> Kliniki - Politechnika Wrocławska, 13:03:00 - 13:05:00, Line: 146
Kliniki - Politechnika Wrocławska -> PL. GRUNWALDZKI, 13:05:00 - 13:07:00, Line: 146
PL. GRUNWALDZKI -> Kochanowskiego, 13:07:00 - 13:10:00, Line: D
Kochanowskiego -> Śniadeckich, 13:10:00 - 13:11:00, Line: D
Śniadeckich -> Zacisze, 13:11:00 - 13:12:00, Line: D
Zacisze -> Kwidzyńska, 13:12:00 - 13:14:00, Line: D
Kwidzyńska -> Brücknera, 13:14:00 - 13:16:00, Line: D
Brücknera -> Psie Pole, 13:16:00 - 13:20:00, Line: D
Time of trip: 13:01:00 - 13:20:00(19 minutes)


datetime.time(13, 20)

# NOW BASED ON DISTANCE

### EDGE : {'arrival_time': datetime.time(22, 52), 'departure_time': datetime.time(22, 50), 'edge_id': 63, 'line': 'A'}

### In case with distance situation is different compared to case with time, because in case with time for each connection(from point a to b for example) we need to take only single min connection, but in the case with lines, we can not do that(cause in case of connections even the worst looking line can become the best one, so I need to keep all possible options inside the queue

## so idea is to separate w dictionary into smaller sub dictionaries. So I will separate it by line. In that way I will be able to reuse : find_minimal_time_difference_edge for each line. For now it might be slower, but it will give me more code readability and Reusability

In [47]:
def separate_by_line(data):
    result = {}
    for key, value in data.items():
        line = value['line']
        if line not in result:
            result[line] = {}
        result[line][key] = value
    return result

In [48]:
def calculate_transfer_cost(w, current_time, current_line):
    result_dict = {}
    separated_by_line = separate_by_line(w)
    for line_name, line_dict in separated_by_line.items():
        min_edge,time_difference = find_minimal_time_difference_edge(line_dict,current_time)
        # if line is different add additional cost ( 1440 because it is one day in fact)
        cost = time_difference if current_line == line_name else time_difference + 1440
        result_dict[line_name] = {'min_edge' : w[min_edge]['edge_id'] , "cost" : cost}
    return result_dict


### EACH NODE WILL BE UNIQUELY IDENTIFIED NOT ONLY BY STOP NAME BUT ALSO LINE (stop_name,line_name)

In [49]:
def astar_path_transfer(G, source, target, start_time, heuristic=None,start_line = None):
    # start time is provided as a string so i need to transform it into datetime.time object
    start_time = str_to_time(start_time)
    if source not in G or target not in G:
        msg = f"Either source {source} or target {target} is not in G"
        raise nx.NodeNotFound(msg)

    if heuristic is None:
        # The default heuristic is h=0 - same as Dijkstra's algorithm
        def heuristic(u, v):
            return 0
    push = heappush
    pop = heappop
    G_succ = G._adj  # For speed-up (and works for both directed and undirected graphs)
    c = count()
    queue = [(0, next(c), source, 0, None, start_time,None)] # priority,consecutive_id,current_node,distance,parent_connection,current_time,prev_line
    
    enqueued = {}
    explored = {}

    while queue:
        # Pop the smallest item from queue.
        _, __, curnode, dist, parent_connection, current_time,prev_line = pop(queue) # current_time specifies at which time we are at particular node
        # prev line is needed to specify if there was a transfer

        # adjust start_line for the start node so I can reuse it in tabu search 
        current_line = edge_hashmap.get(parent_connection)[2]['line'] if parent_connection is not None else start_line
        
        
        if curnode == target:
            path = [parent_connection]
            stop_name = edge_hashmap.get(parent_connection)[0]
            line = prev_line
             
            while stop_name != source:
                prev_connection = explored[(stop_name,line)][0]
                path.append(prev_connection)
                line = explored[stop_name,line][1]
                stop_name = edge_hashmap.get(prev_connection)[0]
                
                
            path.reverse()
            return path
        
        if (curnode,current_line) in explored.keys():
            # Do not override the parent of starting node
            # Skip bad paths that were enqueued before finding a better one
            qcost, h = enqueued[(curnode,current_line)]
            if qcost < dist:
                continue
        # since we will go through all neighbours of this node we mark it as explored
        
        explored[(curnode,current_line)] = parent_connection,prev_line
        
        for neighbor, w in G_succ[curnode].items():
            # used to find connection with the smallest cost for each line from current node from curnode to neighbour ( it is dictionary, since there are multiple lines using which I can get from curnode to neighbour
            transfer_cost = calculate_transfer_cost(w,current_time,current_line)
            if not transfer_cost:
                continue
            # now for each min_edge_line I need to create new entry in queue
            # result_dict[line_name] = {'min_edge' : min_edge , "cost" : cost}
            for line_name,min_connection_info in transfer_cost.items():
                cost = min_connection_info["cost"]
                min_edge = min_connection_info["min_edge"]
                arrival_time_of_min_edge = edge_hashmap.get(min_edge)[2]['arrival_time']
                ncost = dist + cost
                if (neighbor,line_name) in enqueued:
                    qcost, h = enqueued[(neighbor,line_name)]
                    if qcost <= ncost:
                        continue
                else:
                    h = heuristic(G.nodes[neighbor],  G.nodes[target])
                enqueued[(neighbor,line_name)] = ncost, h 
                push(queue, (ncost + h, next(c), neighbor, ncost, min_edge,arrival_time_of_min_edge,current_line))
                # priority,consecutive_id,current_node,distance,parent_connection,current_time,prev_line ( so i can identify if there was transfer)
    raise nx.NetworkXNoPath(f"Node {target} not reachable from {source}")

In [50]:
start_time = "08:19:00"
result = astar_path_transfer(transportation_system,"Tramwajowa","Psie Pole",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

Tramwajowa -> Hala Stulecia, 08:26:00 - 08:29:00, Line: 345
Hala Stulecia -> Mickiewicza, 08:29:00 - 08:30:00, Line: 345
Mickiewicza -> Witelona, 08:30:00 - 08:31:00, Line: 345
Witelona -> Kochanowskiego, 08:31:00 - 08:32:00, Line: 345
Kochanowskiego -> Śniadeckich, 08:32:00 - 08:35:00, Line: 345
Śniadeckich -> Zacisze, 08:36:00 - 08:37:00, Line: 111
Zacisze -> Kwidzyńska, 08:37:00 - 08:39:00, Line: 111
Kwidzyńska -> Brücknera, 08:39:00 - 08:41:00, Line: 111
Brücknera -> C.H. Korona, 08:41:00 - 08:43:00, Line: 111
C.H. Korona -> Zielna, 08:43:00 - 08:45:00, Line: 111
Zielna -> Psie Pole, 08:45:00 - 08:46:00, Line: 111
Time of trip: 08:19:00 - 08:46:00(27 minutes)


datetime.time(8, 46)

In [51]:
start_time = "08:19:00"
result = astar_path_transfer(transportation_system,"Psie Pole","Tramwajowa",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

Psie Pole -> Zielna, 08:22:00 - 08:23:00, Line: 911
Zielna -> C.H. Korona, 08:23:00 - 08:25:00, Line: 911
C.H. Korona -> Brücknera, 08:26:00 - 08:28:00, Line: 911
Brücknera -> Kwidzyńska, 08:28:00 - 08:30:00, Line: 911
Kwidzyńska -> Zacisze, 08:30:00 - 08:31:00, Line: 911
Zacisze -> Śniadeckich, 08:31:00 - 08:32:00, Line: 911
Śniadeckich -> Kochanowskiego, 08:32:00 - 08:35:00, Line: 911
Kochanowskiego -> PL. GRUNWALDZKI, 08:35:00 - 08:37:00, Line: 911
PL. GRUNWALDZKI -> Kliniki - Politechnika Wrocławska, 08:37:00 - 08:39:00, Line: 146
Kliniki - Politechnika Wrocławska -> Hala Stulecia, 08:39:00 - 08:41:00, Line: 146
Hala Stulecia -> Tramwajowa, 08:41:00 - 08:43:00, Line: 146
Time of trip: 08:19:00 - 08:43:00(24 minutes)


datetime.time(8, 43)

In [52]:
start_time = "08:19:00"
result = astar_path_transfer(transportation_system,"DWORZEC GŁÓWNY","most Grunwaldzki",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

DWORZEC GŁÓWNY -> Pułaskiego, 08:21:00 - 08:23:00, Line: 4
Pułaskiego -> Kościuszki, 08:23:00 - 08:25:00, Line: 4
Kościuszki -> Komuny Paryskiej, 08:25:00 - 08:26:00, Line: 4
Komuny Paryskiej -> pl. Wróblewskiego, 08:26:00 - 08:28:00, Line: 4
pl. Wróblewskiego -> Urząd Wojewódzki (Impart), 08:28:00 - 08:31:00, Line: 4
Urząd Wojewódzki (Impart) -> most Grunwaldzki, 08:31:00 - 08:33:00, Line: 4
Time of trip: 08:19:00 - 08:33:00(14 minutes)


datetime.time(8, 33)

In [53]:
start_time = "08:19:00"
result = astar_path_transfer(transportation_system,"DWORZEC GŁÓWNY","Psie Pole",start_time,euclidean_distance)
display_edges_by_id(result,edge_hashmap,start_time)

DWORZEC GŁÓWNY -> GALERIA DOMINIKAŃSKA, 08:19:00 - 08:22:00, Line: N
GALERIA DOMINIKAŃSKA -> Urząd Wojewódzki (Muzeum Narodowe), 08:22:00 - 08:25:00, Line: N
Urząd Wojewódzki (Muzeum Narodowe) -> Katedra, 08:25:00 - 08:26:00, Line: N
Katedra -> Ogród Botaniczny, 08:26:00 - 08:28:00, Line: N
Ogród Botaniczny -> Wyszyńskiego, 08:28:00 - 08:30:00, Line: N
Wyszyńskiego -> Damrota, 08:30:00 - 08:31:00, Line: N
Damrota -> KROMERA, 08:31:00 - 08:34:00, Line: N
KROMERA -> Kromera (Czajkowskiego), 08:34:00 - 08:35:00, Line: N
Kromera (Czajkowskiego) -> Grudziądzka, 08:35:00 - 08:36:00, Line: N
Grudziądzka -> Brücknera, 08:36:00 - 08:37:00, Line: N
Brücknera -> Psie Pole, 08:37:00 - 08:41:00, Line: N
Time of trip: 08:19:00 - 08:41:00(22 minutes)


datetime.time(8, 41)

In [54]:
start_time = "23:45:00"
result = astar_path_transfer(transportation_system,"DWORZEC GŁÓWNY","Psie Pole",start_time,euclidean_distance)
display_edges_by_id(result,edge_hashmap,start_time)

DWORZEC GŁÓWNY -> Wzgórze Partyzantów, 00:04:00 - 00:06:00, Line: 251
Wzgórze Partyzantów -> GALERIA DOMINIKAŃSKA, 00:06:00 - 00:08:00, Line: 251
GALERIA DOMINIKAŃSKA -> Hala Targowa, 00:08:00 - 00:09:00, Line: 251
Hala Targowa -> Dubois, 00:09:00 - 00:11:00, Line: 251
Dubois -> pl. Bema, 00:11:00 - 00:13:00, Line: 251
pl. Bema -> Na Szańcach, 00:13:00 - 00:13:00, Line: 251
Na Szańcach -> Jedności Narodowej, 00:13:00 - 00:14:00, Line: 251
Jedności Narodowej -> Nowowiejska, 00:14:00 - 00:16:00, Line: 251
Nowowiejska -> Daszyńskiego, 00:16:00 - 00:17:00, Line: 251
Daszyńskiego -> Mosty Warszawskie, 00:17:00 - 00:18:00, Line: 251
Mosty Warszawskie -> KROMERA, 00:18:00 - 00:20:00, Line: 251
KROMERA -> Kromera (Czajkowskiego), 00:20:00 - 00:21:00, Line: 251
Kromera (Czajkowskiego) -> Grudziądzka, 00:21:00 - 00:22:00, Line: 251
Grudziądzka -> Brücknera, 00:22:00 - 00:23:00, Line: 251
Brücknera -> C.H. Korona, 00:23:00 - 00:24:00, Line: 251
C.H. Korona -> Zielna, 00:24:00 - 00:26:00, Line: 25

datetime.time(0, 27)

In [55]:
start_time = "08:19:00"
result = astar_path_transfer(transportation_system,"DWORZEC GŁÓWNY","most Grunwaldzki",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

DWORZEC GŁÓWNY -> Pułaskiego, 08:21:00 - 08:23:00, Line: 4
Pułaskiego -> Kościuszki, 08:23:00 - 08:25:00, Line: 4
Kościuszki -> Komuny Paryskiej, 08:25:00 - 08:26:00, Line: 4
Komuny Paryskiej -> pl. Wróblewskiego, 08:26:00 - 08:28:00, Line: 4
pl. Wróblewskiego -> Urząd Wojewódzki (Impart), 08:28:00 - 08:31:00, Line: 4
Urząd Wojewódzki (Impart) -> most Grunwaldzki, 08:31:00 - 08:33:00, Line: 4
Time of trip: 08:19:00 - 08:33:00(14 minutes)


datetime.time(8, 33)

In [56]:
start_time = "08:19:00"
result = astar_path_transfer(transportation_system,"Tramwajowa","most Grunwaldzki",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

Tramwajowa -> ZOO, 08:21:00 - 08:23:00, Line: 4
ZOO -> Hala Stulecia, 08:23:00 - 08:24:00, Line: 4
Hala Stulecia -> Kliniki - Politechnika Wrocławska, 08:24:00 - 08:26:00, Line: 4
Kliniki - Politechnika Wrocławska -> PL. GRUNWALDZKI, 08:26:00 - 08:28:00, Line: 4
PL. GRUNWALDZKI -> most Grunwaldzki, 08:28:00 - 08:29:00, Line: 4
Time of trip: 08:19:00 - 08:29:00(10 minutes)


datetime.time(8, 29)

In [57]:
start_time = "08:19:00"
result = astar_path_transfer(transportation_system,"GALERIA DOMINIKAŃSKA","Zacisze",start_time,manhattan_distance)
display_edges_by_id(result,edge_hashmap,start_time)

GALERIA DOMINIKAŃSKA -> Urząd Wojewódzki (Impart), 08:33:00 - 08:36:00, Line: D
Urząd Wojewódzki (Impart) -> most Grunwaldzki, 08:36:00 - 08:37:00, Line: D
most Grunwaldzki -> PL. GRUNWALDZKI, 08:37:00 - 08:39:00, Line: D
PL. GRUNWALDZKI -> Kochanowskiego, 08:39:00 - 08:42:00, Line: D
Kochanowskiego -> Śniadeckich, 08:42:00 - 08:44:00, Line: D
Śniadeckich -> Zacisze, 08:44:00 - 08:45:00, Line: D
Time of trip: 08:19:00 - 08:45:00(26 minutes)


datetime.time(8, 45)

# CREATE COMBINED METHOD SO IT WILL BE EASIER TO ENTER DATA
Application should accept input in form of 4 variables:
(a) the starting stop A
(b) the ending stop B
(c) optimization criterion: value t denotes travel time minimization, value
p means minimization of the number of transfers
(d) start time
The solution should print out to the standard output in successive lines
detailed path information including starting stop, ending stop, used line’s
name, starting time, ending time, and on the standard error output the
value of minimized criterion and time required for calculation of the shortest path.

In [58]:
def a_star(start_stop,end_stop,optimization_criterion,start_time,heuristic = None,current_line = None):
    result = []
    if optimization_criterion == "t":
        result = astar_path(G = transportation_system,source = start_stop, target = end_stop,start_time = start_time,heuristic = heuristic)
    elif optimization_criterion == "p":
        result = astar_path_transfer(G = transportation_system,source = start_stop, target = end_stop,start_time = start_time,heuristic = heuristic,start_line=current_line) 
    else:
        print("Invalid optimization argument")
    #display_edges_by_id(result,edge_hashmap,start_time)
    return (result)
    

In [59]:
a_star("Tramwajowa","most Grunwaldzki","t","08:19:00",manhattan_distance)

[55713, 55714, 55715, 55716, 107245]

In [60]:
a_star("Borowska (Szpital)","PL. GRUNWALDZKI","p","00:11:00",manhattan_distance)

[912883,
 912884,
 912885,
 912886,
 912887,
 912888,
 912889,
 895309,
 895310,
 895311,
 895312,
 895313]

In [61]:
a_star("Tramwajowa","most Grunwaldzki","t","08:19:00")

[55713, 55714, 55715, 55716, 107245]

In [62]:
a_star("Tramwajowa","most Grunwaldzki","f","08:19:00",manhattan_distance)

Invalid optimization argument


[]

# PART WITH A_STAR AND DIJKSTRA IS FINALLY DONE

# GRID SEARCH TASK
Using the provided file connection_graph.csv implement the shortest
path from a given stop A that visits all the stops given in a list L=
A2, . . . , An, and returns back to A. As a cost function use (depending
on the user’s decision) travel time from A to B or the number of transfers
needed.
The application should accept input consisting of 4 variables:
(a) starting stop A
(b) list of stops to visit separated by a semicolon L
(c) optimization criterion: value t denotes travel time minimization, value
p means minimization of the number of transfers
(d) starting time
The solution should print out to the standard output, in consecutive lines,
the detailed path information including starting stop, ending stop, used
line’s name, starting time, ending time, and on the standard error output
the value of the minimized criterion and time required for calculation of
the shortest path.
Scoring:
(a) an algorithm searching for the shortest path between the nodes based
on Tabu Search without bounding the size of T (10 points)
(b) modification of the algorithm (a) with a variable T size, based on the
length of L, in order to minimize the cost function (5 points)
(c) modification of the algorithm (a) by adding Aspiration criterion, in
order to minimize the cost function (5 points)
(d) modification of the algorithm (a) by adding a neighbor sampling strategy to minimize the cost function and reduce the computation time.
(10 points)


In [63]:
def extract_station_names(input_string):
    # Split the input string based on semicolon delimiter
    station_list = input_string.split(';')
    # Return the list of station names
    return [station.strip() for station in station_list]

In [64]:
def display_answer_tabu_search(connections,start_time):
    current_time = start_time
    for connection in connections:
        current_time = display_edges_by_id(connection,edge_hashmap,current_time).strftime("%H:%M:%S")
        print(current_time)
        

In [66]:
import copy

class TabuSearch:
    def __init__(self, station_names, start, time, tabu_list_size=10, max_iterations=1000,start_line = None,heuristic = None):
        self.start = start
        self.time = time
        self.initial_solution = extract_station_names(station_names)
        self.tabu_list_size = tabu_list_size
        self.max_iterations = max_iterations
        self.max_sol_set = math.factorial(len(station_names))
        self.precalculated_solution = {}
        self.use_aspiration = False # for now i do not use aspiration
        self.start_line = start_line
        self.heuristic = heuristic

    def set_tabu_list_size(self):
        self.tabu_list_size = self.max_sol_set // 2 

    def aspiration(self, iteration):
        return self.use_aspiration and iteration % self.tabu_list_size == 0
    
    def process_result(self,result,start_time,optimization_criterion,current_line):
        last_connection = edge_hashmap.get(result[-1])
        end_time = last_connection[2]['arrival_time']
        end_line = last_connection[2]['line']
        transfer_count = 0
        if optimization_criterion == "p":
            for edge_id_to_find in result:
                edge = edge_hashmap.get(edge_id_to_find)
                line = edge[2]['line']
                if line != current_line:
                    current_line = line
                    transfer_count +=1
            cost = transfer_count
        elif optimization_criterion == "t":
            time_difference = calculate_time_difference(str_to_time(start_time),end_time)      
            cost = time_difference
        else:
            raise ValueError("Optimization criterion must be either 't' or 'p'")
        return cost,end_time,end_line
               
    # run consecutive a star
    def evaluate_solution(self, solution, type,current_line):
        #may mean total time difference or total bus-stop changes
        total_cost = 0
        line_name = current_line
        solution_set =  []
        current_time = self.time
        
        for i in range(len(solution) - 1):
            current_node = solution[i]
            next_node = solution[(i + 1) % len(solution)]
            # to not recalculate each time, save connection weights in dictionary and then extract it 
            # time is needed since from a to b depends on time
            
            
            result_a_star = a_star(current_node,next_node,type,current_time,self.heuristic,line_name) # self.time should be start time 
            cost,end_time,end_line = self.process_result(result_a_star,current_time,type,line_name)
            solution_set.append(result_a_star)
            total_cost += cost
            end_time_str = end_time.strftime("%H:%M:%S")
            current_time = end_time_str
            line_name = end_line
        return total_cost,solution_set

    # is in fact used to prepare all permutations of initial_solution
    def get_neighbors(self, solution: list[str]):
        neighbors= []
        for i in range(len(solution)):
            for j in range(i + 1, len(solution)):
                neighbor = copy.deepcopy(solution)
                neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
                neighbors.append(neighbor)
        return neighbors

    def tabu_search(self, type):
        current_solution = self.initial_solution
        best_solution = current_solution
        best_iteration = 0
        tabu_list = []
        iterations = 0

        while iterations < self.max_iterations:
            neighbors = self.get_neighbors(current_solution)
            best_neighbor = None
            best_neighbor_cost = float('inf')

            for neighbor in neighbors:
                neighbor_cost, _ = self.evaluate_solution([self.start] + neighbor + [self.start], type,self.start_line)
                if neighbor not in tabu_list and neighbor_cost < best_neighbor_cost:
                    best_neighbor = neighbor
                    best_neighbor_cost = neighbor_cost

            if best_neighbor:
                current_solution = best_neighbor
                tabu_list.append(best_neighbor)
                if len(tabu_list) > self.tabu_list_size:
                    tabu_list.pop(0)
                if self.aspiration(iterations):
                    tabu_list = []


                if best_neighbor_cost < self.evaluate_solution([self.start] + best_solution + [self.start], type,self.start_line)[0]:
                    best_solution = best_neighbor
    
            iterations += 1
        return best_solution, self.evaluate_solution([self.start] + best_solution + [self.start], type,self.start_line)


# Initial solution
station_names = "PL. GRUNWALDZKI;Borowska (Szpital);Wzgórze Partyzantów"

# Create TabuSearch object and run tabu search
tabu_search = TabuSearch(station_names, "Tramwajowa", "08:00:00", max_iterations = 3,heuristic = manhattan_distance)
tabu_search.set_tabu_list_size()
best_solution, best_cost = tabu_search.tabu_search("t")

print("Best Solution:", best_solution)
print("Best Cost:", best_cost[0])
display_answer_tabu_search(best_cost[1],"08:00:00")

Best Solution: ['PL. GRUNWALDZKI', 'Wzgórze Partyzantów', 'Borowska (Szpital)']
Best Cost: 71.0
Tramwajowa -> Hala Stulecia, 08:00:00 - 08:02:00, Line: 145
Hala Stulecia -> Kliniki - Politechnika Wrocławska, 08:02:00 - 08:04:00, Line: 145
Kliniki - Politechnika Wrocławska -> PL. GRUNWALDZKI, 08:04:00 - 08:06:00, Line: 145
Time of trip: 08:00:00 - 08:06:00(6 minutes)
08:06:00
PL. GRUNWALDZKI -> most Grunwaldzki, 08:06:00 - 08:07:00, Line: 145
most Grunwaldzki -> Poczta Główna, 08:07:00 - 08:09:00, Line: 145
Poczta Główna -> skwer Krasińskiego, 08:09:00 - 08:11:00, Line: 145
skwer Krasińskiego -> Wzgórze Partyzantów, 08:14:00 - 08:15:00, Line: 149
Time of trip: 08:06:00 - 08:15:00(9 minutes)
08:15:00
Wzgórze Partyzantów -> DWORZEC GŁÓWNY, 08:15:00 - 08:17:00, Line: 5
DWORZEC GŁÓWNY -> DWORZEC AUTOBUSOWY, 08:20:00 - 08:22:00, Line: 9
DWORZEC AUTOBUSOWY -> Dyrekcyjna, 08:22:00 - 08:24:00, Line: 145
Dyrekcyjna -> Borowska (Aquapark), 08:24:00 - 08:26:00, Line: 145
Borowska (Aquapark) -> Śli